#CVPR 2024, Lab class #1

## Estimating homography
In this example, we will estimate homographies from a set of checkerboard images. Estimating homographies is the first step of the Zhang's calibration method.

### Libraries
Besides [Numpy](https://numpy.org/) and the Pyplot interface to [Matplotlib](https://matplotlib.org/), we will need [OpenCV](https://opencv.org/).

In [1]:
from matplotlib import pyplot as plt

import os

import numpy as np

import cv2 # OpenCV
print (cv2.__version__)

4.10.0


Make sure to check the docs for the **your version** of OpenCV, for example, if the installed version is 4.8.0, then the documentation can be found here: https://docs.opencv.org/4.8.0/

### Loading and displaying images using OpenCV
Let the images be located in the `images/` folder. Notice that it may be a folder in your GDrive or even **a shortcut to a folder that is shared with you**.

In [1]:
folderpath = '/images'

1. Calcolare omografia per ogni immagine e c'è già la funzione di Edi

In [ ]:
grid_size = (8,11)
def calculate_homography(image)->np.ndarray:
    return_value, corners = cv2.findChessboardCorners(image, patternSize=grid_size)
    assert(return_value)
    gray = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)
    corners = corners.reshape((88,2)).copy()
    criteria = (cv2.TERM_CRITERIA_MAX_ITER | cv2.TERM_CRITERIA_EPS, 100, 0.001)
    cv2.cornerSubPix(gray, corners, (5,5), (-1,-1), criteria)
    A = np.empty((0,9), dtype=float)
    O = np.array([0,0,0]).reshape(1,3)
    square_size = 11 # mm
    for index, corner in enumerate(corners):
        Xpix = corner[0] # in pixel
        Ypix = corner[1]
        grid_size_cv2 = tuple(reversed(grid_size)) # OpenCV and Python store matrices differently
        u_index, v_index = np.unravel_index(index, grid_size_cv2) # convert index from linear to 2D (0-based indexing)
        Xmm = u_index * square_size
        Ymm = v_index * square_size
        m = np.array([Xmm, Ymm, 1]).reshape(1,3)
        A = np.vstack((A, np.hstack((m,O,-Xpix*m))))
        A = np.vstack((A, np.hstack((O,m,-Ypix*m))))

    _, _, Vh = np.linalg.svd(A)
    h = Vh.transpose()[:,-1]
    H = h.reshape(3,3)
    return H

In [ ]:
images_path = [os.path.join(folderpath, imagename) for imagename in os.listdir(folderpath) if imagename.endswith(".tiff")]
images_path.sort()

In [ ]:
homographies = []
for path in images_path:
    image = cv2.imread(path)
    H = calculate_homography(image)
    homographies.append(H)

2. per ogni omografia dalla relativa matrice costruire la matrice V + fare funzione che costruisce i vettori v, i, j

3. data V trovare B applicando la singular value decomposition

4. da B ottenere K e di conseguenza trovare R